In [ ]:
## Part 1

In [ ]:
## manual code ##

In [2]:
import re
import os
import csv
import glob

import pandas as pd
import numpy as np
from datetime import datetime

import scipy.signal as signal
from scipy.stats import linregress

In [10]:


path = r"./hrv_data/"
all_files = glob.glob(path + "*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep=';') #index_col=None, header=1
    patient_number = int(re.search(r'(\d+)', filename).group(1)) 
    df['ID'] = patient_number
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

df.rename(columns={'TIMESTAMP': 'TIME'}, inplace=True, errors="raise")
# Convert time column to datetime
df['TIME'] = pd.to_datetime(df['TIME'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
#sort df by patient ID
df.sort_values(by="ID", inplace=True)

# Function to calculate features
def calculate_features(group):
    # Calculate PSD
    frequencies, psd = signal.welch(group["HRV"], fs=2, nperseg=128, noverlap=0)

    # Define frequency bands
    vlf_band = (0.003, 0.04)
    lf_band = (0.04, 0.15)
    hf_band = (0.15, 0.4)

    # Calculate power in each band
    vlf_power = np.sum(psd[(frequencies >= vlf_band[0]) & (frequencies < vlf_band[1])])
    lf_power = np.sum(psd[(frequencies >= lf_band[0]) & (frequencies < lf_band[1])])
    hf_power = np.sum(psd[(frequencies >= hf_band[0]) & (frequencies < hf_band[1])])

    # Calculate LF/HF ratio
    lf_hf_ratio = lf_power / hf_power if hf_power != 0 else np.nan

    # Find peak frequency and total power
    peak_frequency = frequencies[np.argmax(psd)]
    total_power = np.sum(psd)

    nn_intervals = group["HRV"].diff()
    nn50 = (nn_intervals.abs() > 50).sum()
    pnn50 = nn50 / len(nn_intervals) if len(nn_intervals) > 0 else 0
    slope, intercept, r_value, p_value, std_err = linregress(
        group["TIME"].astype(int) / 10**9, group["HRV"]
    )

    mean_hrv = group["HRV"].mean()
    
    return pd.Series(
        {
            "mean_HRV": mean_hrv,
            "std_HRV": group["HRV"].std(),
            "min_HRV": group["HRV"].min(),
            "max_HRV": group["HRV"].max(),
            "25th_percentile_HRV": np.percentile(group["HRV"], 25),
            "50th_percentile_HRV": np.percentile(group["HRV"], 50),
            "75th_percentile_HRV": np.percentile(group["HRV"], 75),
            "skewness_HRV": group["HRV"].skew(),
            "kurtosis_HRV": group["HRV"].kurtosis(),
            "above_mean_HRV": (group["HRV"] > mean_hrv).sum(),
            "below_mean_HRV": (group["HRV"] < mean_hrv).sum(),
            "variance_HRV": group["HRV"].var(),
            "range_HRV": group["HRV"].max() - group["HRV"].min(),
            "iqr_HRV": np.percentile(group["HRV"], 75) - np.percentile(group["HRV"], 25),
            "mad_HRV": (group["HRV"] - group["HRV"].mean()).abs().mean(),
            "medad_HRV": (group["HRV"] - group["HRV"].median()).abs().median(),
            "rms_HRV": np.sqrt(np.mean(group["HRV"]**2)),
            "cv_HRV": group["HRV"].std() / group["HRV"].mean() if group["HRV"].mean() != 0 else 0,
            "md_HRV": group["HRV"].diff().mean(),
            "msd_HRV": group["HRV"].diff().diff().mean(),
            "rmssd_HRV": np.sqrt(np.mean(group["HRV"].diff()**2)),
            "vlf_power_HRV": vlf_power,
            "lf_power_HRV": lf_power,
            "hf_power_HRV": hf_power,
            "lf_hf_ratio_HRV": lf_hf_ratio,
            "peak_frequency_HRV": peak_frequency,
            "total_power_HRV": total_power,
            "nn50_HRV": nn50,
            "pnn50_HRV": pnn50,
            "slope_time_HRV": slope,
            "intercept_time_HRV": intercept,
            "correlation_time_HRV": r_value,
            "p_value_correlation_time_HRV": p_value,
            "std_err_slope_time_HRV": std_err,
            #"std_err_intercept_HRV": np.nan,  # Not provided by linregress
        }
    )

# Group by patient_id and apply the function
# For include_groups=False check https://stackoverflow.com/questions/77969964/deprecation-warning-with-groupby-apply
features_df = df.groupby("ID").apply(calculate_features, include_groups=False).reset_index()

# Print the final DataFrame
#print(features_df.to_markdown(index=False, numalign="left", stralign="left"))


In [11]:
df.shape

(7742746, 3)

In [12]:
df.head(5)

,TIME,HRV,ID
5895931,2009-03-04 16:44:51.460,721.68,1
5902763,2009-03-04 18:01:33.646,593.75,1
5902764,2009-03-04 18:01:34.240,593.75,1
5902765,2009-03-04 18:01:34.834,605.47,1
5902766,2009-03-04 18:01:35.439,611.33,1


In [17]:
pd.set_option('display.max_columns', None)
features_df.head(5)

,ID,mean_HRV,std_HRV,min_HRV,max_HRV,25th_percentile_HRV,50th_percentile_HRV,75th_percentile_HRV,skewness_HRV,kurtosis_HRV,above_mean_HRV,below_mean_HRV,variance_HRV,range_HRV,iqr_HRV,mad_HRV,medad_HRV,rms_HRV,cv_HRV,md_HRV,msd_HRV,rmssd_HRV,vlf_power_HRV,lf_power_HRV,hf_power_HRV,lf_hf_ratio_HRV,peak_frequency_HRV,total_power_HRV,nn50_HRV,pnn50_HRV,slope_time_HRV,intercept_time_HRV,correlation_time_HRV,p_value_correlation_time_HRV,std_err_slope_time_HRV
0,1,707.652854,139.830794,323.24,2697.27,638.67,701.17,756.84,5.456632,48.433579,50197.0,57277.0,19552.650890,2374.03,118.17,78.230348,58.60,721.335588,0.197598,-0.000064,0.001554,81.317593,164386.041501,87184.660476,57012.186631,1.529228,0.015625,4.437704e+05,10207.0,0.094972,-0.000170,2.103045e+05,-0.026657,2.318335e-18,0.000019
1,3,759.529730,158.316375,191.41,2469.73,645.51,740.23,859.38,2.102284,12.322806,43953.0,55059.0,25064.074447,2278.32,213.87,117.626179,103.51,775.853873,0.208440,0.001401,-0.000237,133.390469,72696.337020,139416.269074,187781.190420,0.742440,0.015625,7.515955e+05,15367.0,0.155203,0.002678,-3.306995e+06,0.376565,0.000000e+00,0.000021
2,4,768.352224,302.064748,108.40,2892.58,646.48,839.84,988.28,-0.663971,-0.440612,60893.0,41230.0,91243.112165,2784.18,341.80,242.300340,162.11,825.595154,0.393133,0.003643,-0.000918,104.206084,302077.041635,251265.927903,157093.855870,1.599464,0.015625,9.560329e+05,18961.0,0.185668,-0.006275,7.766233e+06,-0.499847,0.000000e+00,0.000034
3,5,1003.211186,486.852776,188.48,3062.50,689.45,825.20,998.05,1.368901,0.451574,18907.0,58111.0,237025.625059,2874.02,308.60,369.311739,147.47,1115.103238,0.485294,0.005389,-0.053471,259.989322,592368.066500,572244.933233,703451.749918,0.813481,0.015625,3.199768e+06,25679.0,0.333416,0.016910,-2.089109e+07,0.715116,0.000000e+00,0.000060
4,7,828.531332,188.033565,192.38,3033.20,726.56,833.98,922.85,1.399269,9.478426,47719.0,45002.0,35356.621589,2840.82,196.29,130.216321,96.68,849.600146,0.226948,-0.003792,-0.000021,142.290372,236369.477738,264338.597443,222101.121533,1.190172,0.015625,1.117356e+06,27560.0,0.297236,0.002534,-3.133269e+06,0.295416,0.000000e+00,0.000027


In [14]:
features_df.shape

(80, 35)

In [15]:
# save features files
path = r"./data_output/"
file = "features_manual.csv"

features_df.to_csv(path+file, sep=';', encoding='utf-8', index=False, header=True)

In [ ]:
## Part 1

In [ ]:
## tsfresh code ##

In [1]:
import re
import os
import csv
import glob

import pandas as pd
from datetime import datetime

from tsfresh import extract_features
from tsfresh import select_features

from tsfresh.utilities.dataframe_functions import impute
from tsfresh.utilities.dataframe_functions import roll_time_series

In [2]:


path = r"./hrv_data/"
all_files = glob.glob(path + "*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep=';') #index_col=None, header=1
    patient_number = int(re.search(r'(\d+)', filename).group(1))
    df['ID'] = patient_number
    li.append(df)

# combine all files data into one df
df = pd.concat(li, axis=0, ignore_index=True)

In [3]:
# tidy dataframe

df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')

df.sort_values(['ID', 'TIMESTAMP'], ascending=[True, True], inplace=True)

df.reset_index(drop=True, inplace=True)

In [6]:
df.shape

(7742746, 3)

In [7]:
df.head(5)

,TIMESTAMP,HRV,ID
0,2009-03-04 11:00:00.000,2294.92,1
1,2009-03-04 11:00:02.295,631.84,1
2,2009-03-04 11:00:02.927,624.02,1
3,2009-03-04 11:00:03.551,636.72,1
4,2009-03-04 11:00:04.188,625.98,1


In [8]:
df.dtypes

TIMESTAMP    datetime64[ns]
HRV                 float64
ID                    int64
dtype: object

In [10]:
# slice df 5 rows
x = df[0:5]
x.head()

,TIMESTAMP,HRV,ID
0,2009-03-04 11:00:00.000,2294.92,1
1,2009-03-04 11:00:02.295,631.84,1
2,2009-03-04 11:00:02.927,624.02,1
3,2009-03-04 11:00:03.551,636.72,1
4,2009-03-04 11:00:04.188,625.98,1


In [12]:
# produce minimal features only, due to impossible job to handle size and calculation overhead for full 788 feature set on 18m records locally.
from tsfresh.feature_extraction import EfficientFCParameters, MinimalFCParameters

extracted_features = extract_features(df, column_id="ID", column_sort="TIMESTAMP",  default_fc_parameters=MinimalFCParameters())

Feature Extraction: 100%|███████████████████████| 40/40 [00:04<00:00,  8.46it/s]


In [13]:
extracted_features.shape

(80, 10)

In [16]:
extracted_features.head()

,HRV__sum_values,HRV__median,HRV__mean,HRV__length,HRV__standard_deviation,HRV__variance,HRV__root_mean_square,HRV__maximum,HRV__absolute_maximum,HRV__minimum,ID
1,76054282.86,701.17,707.652854,107474.0,139.830143,19552.468961,721.335588,2697.27,2697.27,323.24,1
3,75202557.64,740.23,759.529730,99012.0,158.315575,25063.821306,775.853873,2469.73,2469.73,191.41,3
4,78466434.18,839.84,768.352224,102123.0,302.063269,91242.218702,825.595154,2892.58,2892.58,108.40,4
5,77265319.09,825.20,1003.211186,77018.0,486.849615,237022.547524,1115.103238,3062.50,3062.50,188.48,5
7,76822253.59,833.98,828.531332,92721.0,188.032551,35356.240267,849.600146,3033.20,3033.20,192.38,7


In [18]:
extracted_features.drop('ID', axis=1, inplace=True, errors='ignore')
extracted_features.insert(0,'ID','')
extracted_features['ID'] = extracted_features.index

In [19]:
extracted_features.head()

,ID,HRV__sum_values,HRV__median,HRV__mean,HRV__length,HRV__standard_deviation,HRV__variance,HRV__root_mean_square,HRV__maximum,HRV__absolute_maximum,HRV__minimum
1,1,76054282.86,701.17,707.652854,107474.0,139.830143,19552.468961,721.335588,2697.27,2697.27,323.24
3,3,75202557.64,740.23,759.529730,99012.0,158.315575,25063.821306,775.853873,2469.73,2469.73,191.41
4,4,78466434.18,839.84,768.352224,102123.0,302.063269,91242.218702,825.595154,2892.58,2892.58,108.40
5,5,77265319.09,825.20,1003.211186,77018.0,486.849615,237022.547524,1115.103238,3062.50,3062.50,188.48
7,7,76822253.59,833.98,828.531332,92721.0,188.032551,35356.240267,849.600146,3033.20,3033.20,192.38


In [20]:
feature_filepath= './data_output/features_tsfresh_minimal.csv'
extracted_features.to_csv(feature_filepath, sep=';', encoding='utf-8', index=False, header=True)

In [ ]:
# Part 3

In [ ]:
## combine manual and tsfresh features ##

In [22]:
## combine manual and tsfresh features into one features file ###
dir = r"./data_output/"
file = "features_tsfresh_minimal.csv"
df_tsfresh_minimal = pd.read_csv(dir+file, sep=';')

path = r"./data_output/"
file = "features_manual.csv"
df_manual = pd.read_csv(dir+file, sep=';')

In [23]:
df_tsfresh_minimal.columns

Index(['ID', 'HRV__sum_values', 'HRV__median', 'HRV__mean', 'HRV__length',
       'HRV__standard_deviation', 'HRV__variance', 'HRV__root_mean_square',
       'HRV__maximum', 'HRV__absolute_maximum', 'HRV__minimum'],
      dtype='object')

In [24]:
df_manual.columns

Index(['ID', 'mean_HRV', 'std_HRV', 'min_HRV', 'max_HRV',
       '25th_percentile_HRV', '50th_percentile_HRV', '75th_percentile_HRV',
       'skewness_HRV', 'kurtosis_HRV', 'above_mean_HRV', 'below_mean_HRV',
       'variance_HRV', 'range_HRV', 'iqr_HRV', 'mad_HRV', 'medad_HRV',
       'rms_HRV', 'cv_HRV', 'md_HRV', 'msd_HRV', 'rmssd_HRV', 'vlf_power_HRV',
       'lf_power_HRV', 'hf_power_HRV', 'lf_hf_ratio_HRV', 'peak_frequency_HRV',
       'total_power_HRV', 'nn50_HRV', 'pnn50_HRV', 'slope_time_HRV',
       'intercept_time_HRV', 'correlation_time_HRV',
       'p_value_correlation_time_HRV', 'std_err_slope_time_HRV'],
      dtype='object')

In [29]:
common_list = ['mean_HRV', 'variance_HRV', 'std_HRV', 'min_HRV', 'max_HRV']
thisFilter = df_manual.filter(common_list)
df_manual.drop(thisFilter, inplace=True, axis=1)

In [30]:
df_manual.columns

Index(['ID', '25th_percentile_HRV', '50th_percentile_HRV',
       '75th_percentile_HRV', 'skewness_HRV', 'kurtosis_HRV', 'above_mean_HRV',
       'below_mean_HRV', 'range_HRV', 'iqr_HRV', 'mad_HRV', 'medad_HRV',
       'rms_HRV', 'cv_HRV', 'md_HRV', 'msd_HRV', 'rmssd_HRV', 'vlf_power_HRV',
       'lf_power_HRV', 'hf_power_HRV', 'lf_hf_ratio_HRV', 'peak_frequency_HRV',
       'total_power_HRV', 'nn50_HRV', 'pnn50_HRV', 'slope_time_HRV',
       'intercept_time_HRV', 'correlation_time_HRV',
       'p_value_correlation_time_HRV', 'std_err_slope_time_HRV'],
      dtype='object')

In [38]:
df_combined = df_tsfresh_minimal.merge(df_manual, how='inner', left_on=['ID'], right_on=['ID'] #left_index=True, right_index=True,
                 , suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

In [39]:
df_combined.head(5)

,ID,HRV__sum_values,HRV__median,HRV__mean,HRV__length,HRV__standard_deviation,HRV__variance,HRV__root_mean_square,HRV__maximum,HRV__absolute_maximum,...,lf_hf_ratio_HRV,peak_frequency_HRV,total_power_HRV,nn50_HRV,pnn50_HRV,slope_time_HRV,intercept_time_HRV,correlation_time_HRV,p_value_correlation_time_HRV,std_err_slope_time_HRV
0,1,76054282.86,701.17,707.652854,107474.0,139.830143,19552.468961,721.335588,2697.27,2697.27,...,1.529228,0.015625,4.437704e+05,10207.0,0.094972,-0.000170,2.103045e+05,-0.026657,2.318335e-18,0.000019
1,3,75202557.64,740.23,759.529730,99012.0,158.315575,25063.821306,775.853873,2469.73,2469.73,...,0.742440,0.015625,7.515955e+05,15367.0,0.155203,0.002678,-3.306995e+06,0.376565,0.000000e+00,0.000021
2,4,78466434.18,839.84,768.352224,102123.0,302.063269,91242.218702,825.595154,2892.58,2892.58,...,1.599464,0.015625,9.560329e+05,18961.0,0.185668,-0.006275,7.766233e+06,-0.499847,0.000000e+00,0.000034
3,5,77265319.09,825.20,1003.211186,77018.0,486.849615,237022.547524,1115.103238,3062.50,3062.50,...,0.813481,0.015625,3.199768e+06,25679.0,0.333416,0.016910,-2.089109e+07,0.715116,0.000000e+00,0.000060
4,7,76822253.59,833.98,828.531332,92721.0,188.032551,35356.240267,849.600146,3033.20,3033.20,...,1.190172,0.015625,1.117356e+06,27560.0,0.297236,0.002534,-3.133269e+06,0.295416,0.000000e+00,0.000027


In [40]:
df_combined.columns

Index(['ID', 'HRV__sum_values', 'HRV__median', 'HRV__mean', 'HRV__length',
       'HRV__standard_deviation', 'HRV__variance', 'HRV__root_mean_square',
       'HRV__maximum', 'HRV__absolute_maximum', 'HRV__minimum',
       '25th_percentile_HRV', '50th_percentile_HRV', '75th_percentile_HRV',
       'skewness_HRV', 'kurtosis_HRV', 'above_mean_HRV', 'below_mean_HRV',
       'range_HRV', 'iqr_HRV', 'mad_HRV', 'medad_HRV', 'rms_HRV', 'cv_HRV',
       'md_HRV', 'msd_HRV', 'rmssd_HRV', 'vlf_power_HRV', 'lf_power_HRV',
       'hf_power_HRV', 'lf_hf_ratio_HRV', 'peak_frequency_HRV',
       'total_power_HRV', 'nn50_HRV', 'pnn50_HRV', 'slope_time_HRV',
       'intercept_time_HRV', 'correlation_time_HRV',
       'p_value_correlation_time_HRV', 'std_err_slope_time_HRV'],
      dtype='object')

In [41]:
dir = r"./data_output/"
file = "features_manual_combined.csv"
df_combined.to_csv(dir+file, sep=';', encoding='utf-8', index=False, header=True)